# FaceMask Image Classification
### withMask, withoutMask

In [ ]:
train_dir = "../input/face-mask-12k-images-dataset/Face Mask Dataset/Train/"
valid_dir = "../input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/"
test_dir  = "../input/face-mask-12k-images-dataset/Face Mask Dataset/Test/"

## Data Augmentation

In [ ]:
import numpy as np
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import models, layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
target_size=(96,96)
batch_size = 16

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=False,    
    class_mode='categorical')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb', 
    shuffle=False,    
    class_mode='categorical')

## Build Model

In [ ]:
num_classes = 2 # WithMask, WithoutMask
input_shape = (96,96,3)

In [ ]:
# Build Model
model = models.Sequential()
# 1st Conv layer
model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
# 2nd Conv layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
# 3rd Conv layer
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
# 4th Conv layer
model.add(layers.Conv2D(96, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
# 5th Conv layer
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
# FC layers
model.add(layers.Flatten())
#model.add(layers.Dense(1024, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(layers.Dense(1024))
#model.add(layers.Dropout(0.2))

#model.add(layers.Dense(64, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(layers.Dense(64))
#model.add(layers.Dropout(0.2))

model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='Adam',  metrics=['accuracy'])

## Train Model

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST =test_generator.n//test_generator.batch_size
num_epochs = 50

In [ ]:
# Train Model
model.fit_generator(train_generator,steps_per_epoch=STEP_SIZE_TRAIN,epochs=num_epochs, validation_data=valid_generator, validation_steps=STEP_SIZE_VALID) #, callbacks=[checkpoint])

## Save Model

In [ ]:
models.save_model(model, "facemask_cnn.h5")

## Evaluate Model

In [ ]:
score = model.evaluate_generator(test_generator, steps=STEP_SIZE_TEST)
print(score)

## Confusion Matrix

In [ ]:
predY=model.predict_generator(test_generator)
y_pred = np.argmax(predY,axis=1)
#y_label= [labels[k] for k in y_pred]
y_actual = test_generator.classes
cm = confusion_matrix(y_actual, y_pred)
print(cm)

In [ ]:
# report
labels = ['withMask', 'withoutMask']
print(classification_report(y_actual, y_pred, target_names=labels))